In [24]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [46]:
building_type = "one_room"

df = pd.read_csv(f"results/{building_type}/{building_type}_cluster_means.csv",parse_dates=[-1], index_col=[0])
dfT = df.T
dfT.head()

,2010-01-01,2010-04-01,2010-07-01,2010-10-01,2011-01-01,2011-04-01,2011-07-01,2011-10-01,2012-01-01,2012-04-01,...,2019-01-01,2019-04-01,2019-07-01,2019-10-01,2020-01-01,2020-04-01,2020-07-01,2020-10-01,2021-01-01,2021-04-01
0,2180.208333,2166.35,2209.555556,2312.45,2187.0,2364.0,2353.956522,2302.571429,2405.0625,2402.35,...,2383.769231,2550.888889,2602.4375,2472.357143,2630.733333,2741.933333,2568.73913,2590.291667,2779.933333,2778.823529
1,1553.5,1482.181818,1595.076923,1665.2,1718.090909,1705.461538,1703.133333,1570.2,1727.0,1695.727273,...,1837.8,1909.6,1911.777778,1734.0,1796.285714,1951.0,1806.818182,1669.555556,1804.0,1804.0
2,2639.428571,2742.8,2670.3,2828.125,2756.8,2806.923077,2871.272727,2987.875,2919.8,3023.0,...,3256.142857,3692.4,3216.333333,3408.571429,3487.909091,3477.454545,3603.125,3728.461538,3785.909091,3774.071429
3,3107.923077,3292.923077,3382.25,3232.636364,3387.444444,3486.411765,3542.923077,3512.357143,3480.363636,3496.75,...,4320.0,4515.888889,4429.4,4328.125,4629.333333,4508.181818,4648.692308,4776.375,4893.45,5019.692308
4,1064.0,1182.357143,984.625,1050.818182,1099.333333,1085.857143,1142.615385,972.625,1030.0,1171.153846,...,940.0,1078.0,902.333333,1125.8,846.25,1135.75,971.6,1019.928571,995.666667,1032.333333


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as MAE,mean_absolute_percentage_error as MAPE

X,y = dfT.iloc[:,:-4], dfT.iloc[:,-4:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
y_train.head()

,2020-07-01,2020-10-01,2021-01-01,2021-04-01
3,4648.692308,4776.375,4893.45,5019.692308
5,8023.615384615385,7789.6,7832.64705882353,8234.90909090909
4,971.6,1019.928571,995.666667,1032.333333
2,3603.125,3728.461538,3785.909091,3774.071429


In [48]:
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

mae = MAE(y_test.T, y_pred.T, multioutput='raw_values')
mape = MAPE(y_test.T, y_pred.T, multioutput='raw_values')

print(building_type)
print("MAE avg: ", round(np.mean(mae),3), "MAPE avg: ", round(np.mean(mape),3))
print("MAE median: ", round(np.median(mae),3), "MAPE median: ", round(np.median(mape),3))

one_room
MAE avg:  142.601 MAPE avg:  0.067
MAE median:  142.601 MAPE median:  0.067


In [ ]:
from sklearn.linear_model import LinearRegression

res_df = pd.DataFrame(columns=df.columns, index=['pred_0','pred_1','pred_2','pred_3'])



In [15]:
# Basic imports
import numpy as np
import pandas as pd
import datetime # manipulating date formats
import itertools

from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

def sarimax_forecasts(df, max_p=2, d=1, max_q=2, max_P=2, D=1, max_Q=2, s=4):
    """
    Keyword arguments:
    multivar_ts -- pandas dataframe of shape (n,m) where n is the number of
    time steps and n is the number of variables for prediction.
    """
    
    res_df = pd.DataFrame(columns=df.columns, index=['pred_0','pred_1','pred_2','pred_3'])
    # set parameter range
    p,d,q = range(0,max_p+1),[d],range(0,max_q+1)
    P,D,Q,s = range(0,max_P+1),[D],range(0,max_Q+1),[s]

    # list of all parameter combos
    pdq = list(map(list,itertools.product(p, d, q)))
    seasonal_pdq = list(map(list, itertools.product(P, D, Q, s)))
    all_param = list(map(list, itertools.product(pdq,seasonal_pdq)))

    count = 0
    for column in df:
        best_res, best_aic = None, np.inf
        for param in all_param:
            try:
                mod = SARIMAX(
                    df[column].values,
                    order=param[0],
                    seasonal_order=param[1]
                )
                
                res = mod.fit(disp=0)
                if res.aic < best_aic:
                    best_res, best_aic = res, res.aic
            except Exception as e:
                print(e)
                continue
        
        count += 1
        # nice = best_res.forecast(steps=4)
        res_df[column] = best_res.forecast(steps=4)

        print(f'The prediction for {column} is:', res_df[column].values)

        print(f"Processed {round(count/len(df.columns)*100,2)}%: {count}/{len(df.columns)}")
            
    return res_df

In [16]:
sarimax_forecasts(df.iloc[:,[0]])

The prediction for 00100 is: [8686.12097272 8582.49246    8841.90816584 9063.901058  ]
Processed 100.0%: 1/1


,00100
pred_0,8686.120973
pred_1,8582.492460
pred_2,8841.908166
pred_3,9063.901058


In [19]:
df = pd.read_csv("prediction_datasets/one_room/one_room_sarimax_forecasts.csv", index_col=[0])
df.head()

,00100,00120
pred_0,8686.120973,8864.809120
pred_1,8582.492460,8352.647861
pred_2,8841.908166,8739.900611
pred_3,9063.901058,8938.433763
